1. Importing Libraries
2. Preprocessing
3. KRW-BTC candle chart
      - plotly
4. KRW-BTC,SOL,XRP,DOGE daily price returns
      - graph
      - histogram
5. KRW-BTC,SOL,XRP,DOGE Correlation
      - scatter matrix
      - heatmap
6. KRW-BTC,SOL,XRP,DOGE risk and returns
      - scatter 
7. Predicting closing price KRW-BTC
      - LSTM

In [1]:
%load_ext autoreload
%autoreload 2

## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import ta.momentum
import ta.volatility
import ta.volume
import ta.trend
import requests
from ta.utils import dropna

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from modules import *
import time

# Preprocessing

In [2]:
api=UpbitAPI()

In [7]:
df = api.day_candle(count=1500)
def applytech(df):
    df['bol_upper']=ta.volatility.bollinger_hband(df['trade_price'])
    df['bol_lower']=ta.volatility.bollinger_lband(df['trade_price'])
    df['rsi']=ta.momentum.rsi(df['trade_price'])
    df['12ema']=ta.trend.ema_indicator(df['trade_price'],window=12)
    df['26ema']=ta.trend.ema_indicator(df['trade_price'],window=26)
    df['macd']=ta.trend.macd(df['trade_price'])
    df['macd_signal']=ta.trend.macd_signal(df['trade_price'])
    df['macd_oscillators']=ta.trend.macd_diff(df['trade_price'])
    df['Daily return']=df['trade_price'].pct_change()
    df.rename(columns = {'opening_price':'open','trade_price':'close','high_price':'high','low_price':'low','candle_acc_trade_volume':'volume'},inplace=True)
    return df
df=applytech(df)
df

,open,high,low,close,candle_acc_trade_price,volume,bol_upper,bol_lower,rsi,12ema,26ema,macd,macd_signal,macd_oscillators,Daily return
datetime64_kst,,,,,,,,,,,,,,,
2023-01-16 09:00:00,25857000.0,26500000.0,25600000.0,26086000.0,1.112000e+11,4273.707594,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-15 09:00:00,25873000.0,25990000.0,25447000.0,25836000.0,1.225455e+11,4769.184180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.009584
2023-01-14 09:00:00,24802000.0,26201000.0,24766000.0,25881000.0,3.680309e+11,14303.123998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001742
2023-01-13 09:00:00,23546000.0,24902000.0,23378000.0,24802000.0,1.546906e+11,6444.405014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.041691
2023-01-12 09:00:00,22475000.0,23900000.0,22415000.0,23552000.0,1.759627e+11,7650.901631,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.050399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-06 09:00:00,4262000.0,4391000.0,3950000.0,3965000.0,5.974818e+10,14125.618431,4.624739e+06,3.440261e+06,48.163707,3.925227e+06,4.012374e+06,-87147.195649,-90351.848622,3204.652973,0.025078
2018-12-05 09:00:00,4456000.0,4472000.0,4232000.0,4262000.0,3.623557e+10,8325.803427,4.627096e+06,3.439804e+06,57.202959,3.977038e+06,4.030865e+06,-53826.751291,-83046.829156,29220.077865,0.074905
2018-12-04 09:00:00,4393000.0,4592000.0,4280000.0,4457000.0,4.347475e+10,9755.045059,4.609963e+06,3.448737e+06,61.900597,4.050878e+06,4.062430e+06,-11552.024215,-68747.868167,57195.843953,0.045753


## KRW-BTC candle chart


    - plotly

In [8]:
# set plot
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, 
               vertical_spacing=0.02, 
               row_width=[0.3,0.1,0.6])

# row 1, col 1
fig.add_trace(go.Candlestick(x=df.index,
                    open=df['open'], high=df['high'],
                    low=df['low'], close=df['close'],showlegend=False),row=1, col=1,)
fig.add_trace(go.Scatter(x=df.index,y=df['bol_upper'],name='bol_upper',line=dict(color='royalblue', width=1, dash='dash')), row=1,
                 col=1)
fig.add_trace(go.Scatter(x=df.index,y=df['bol_lower'],name='bol_lower',line=dict(color='firebrick', width=1, dash='dash')), row=1,
                 col=1)
fig.add_trace(go.Scatter(x=df.index,y=df['26ema'],name='26ema',line=dict(color='blue', width=1, )), row=1,
                 col=1)
fig.add_trace(go.Scatter(x=df.index,y=df['12ema'],name='12ema',line=dict(color='#ff3399', width=1, )), row=1,
                 col=1)

# row 2, col 1
green_volume_df = df[df['close'] > df['open']]
red_volume_df = df[df['close'] < df['open']]
fig.add_trace(go.Bar(x=red_volume_df.index, y=red_volume_df.volume, showlegend=False, marker_color='#ef5350'), row=2,
                 col=1)
fig.add_trace(go.Bar(x=green_volume_df.index, y=green_volume_df.volume, showlegend=False, marker_color='#26a69a'),
                 row=2, col=1)

# row 3, col 1
fig.add_trace(go.Scatter(x=df.index,y=df['rsi'],name='rsi',line=dict(color='blue', width=1, )), row=3,
                 col=1)
fig.add_trace(go.Bar(x=df.index,y=df['macd_oscillators'],name='macd_osc[12,26,9]'), row=3,
                 col=1)
fig.add_trace(go.Scatter(x=df.index,y=df['macd'],name='macd',), row=3,
                 col=1)
fig.add_trace(go.Scatter(x=df.index,y=df['macd_signal'],name='macd_signal',), row=3,
                 col=1)

# label and plot
fig.update(layout_xaxis_rangeslider_visible=False)
fig.update_layout(title=f'KRW-BTC', yaxis_title=f'Price')
fig.update_yaxes(title_text=f'Volume', row=2, col=1)
fig.update_yaxes(title_text=f'MACD', row=3, col=1)
fig.update_xaxes(title_text='Date', row=2)
fig.show()

## KRW-BTC,SOL,XRP,DOGE daily price return
    - graph

In [15]:
# get data
df_BTC=applytech(api.min_candle(unit=10,market="KRW-BTC",count=1000))

df_SOL=applytech(api.min_candle(unit=10,market="KRW-SOL",count=1000))

df_XRP=applytech(api.min_candle(unit=10,market="KRW-XRP",count=1000))

df_DOGE=applytech(api.min_candle(unit=10,market="KRW-DOGE",count=1000))


# x-axis by time series
fig=make_subplots(rows=2, cols=2, vertical_spacing=0.02)

fig.add_trace(go.Scatter(x=df_BTC.index,y=df_BTC['Daily return'],name='KRW_BTC',line=dict(color='royalblue', width=0.5)), row=1,
                 col=1)
fig.add_trace(go.Scatter(x=df_SOL.index,y=df_SOL['Daily return'],name='KRW_SOL',line=dict(color='royalblue', width=0.5)), row=1,
                 col=2)
fig.add_trace(go.Scatter(x=df_XRP.index,y=df_XRP['Daily return'],name='KRW_XRP',line=dict(color='royalblue', width=0.5)), row=2,
                 col=1)
fig.add_trace(go.Scatter(x=df_DOGE.index,y=df_DOGE['Daily return'],name='KRW_DOGE',line=dict(color='royalblue', width=0.5)), row=2,
                 col=2)

fig.update(layout_xaxis_rangeslider_visible=False)
fig.update_layout(title=f'Daily return', yaxis_title=f'pct_change')
fig.update_yaxes(title_text=f'KRW_BTC', row=1, col=1)
fig.update_yaxes(title_text=f'KRW_SOL', row=1, col=2)
fig.update_yaxes(title_text=f'KRW_XRP', row=2, col=1)
fig.update_yaxes(title_text=f'KRW_DOGE', row=2, col=2)

    - histogram

In [16]:
# x-axis by values(pct_change)

fig=make_subplots(rows=2, cols=2, vertical_spacing=0.02)

fig.add_trace(go.Histogram(x=df_BTC['Daily return'],name='KRW_BTC'), row=1,
                 col=1)
fig.add_trace(go.Histogram(x=df_SOL['Daily return'],name='KRW_SOL',), row=1,
                 col=2)
fig.add_trace(go.Histogram(x=df_XRP['Daily return'],name='KRW_XRP',), row=2,
                 col=1)
fig.add_trace(go.Histogram(x=df_DOGE['Daily return'],name='KRW_DOGE',), row=2,
                 col=2)

fig.update(layout_xaxis_rangeslider_visible=False)
fig.update_layout(title=f'Daily return', yaxis_title=f'pct_change')
fig.update_yaxes(title_text=f'KRW_BTC', row=1, col=1)
fig.update_yaxes(title_text=f'KRW_SOL', row=1, col=2)
fig.update_yaxes(title_text=f'KRW_XRP', row=2, col=1)
fig.update_yaxes(title_text=f'KRW_DOGE', row=2, col=2)

## KRW-BTC,SOL,XRP,DOGE returns Correlation
    - Scatter matrix

In [17]:
# Splom = scatter matrix in plotly
fig = go.Figure(data=go.Splom(
                dimensions=[dict(label='KRW_BTC',
                                 values=df_BTC['Daily return']),
                            dict(label='KRW_SOL',
                                 values=df_SOL['Daily return']),
                            dict(label='KRW_XRP',
                                 values=df_XRP['Daily return']),
                            dict(label='KRW_DOGE',
                                 values=df_DOGE['Daily return'])],
                )
                )


fig.update_layout(
    title='Daily return',
    dragmode='select',
    width=600,
    height=600,
    hovermode='closest',
)

fig.show()

    - Correlation heatmap

In [19]:
# get correlation matrix
stocks_returns=pd.concat([df_BTC['Daily return'],df_SOL['Daily return'],df_XRP['Daily return'],df_DOGE['Daily return']],axis=1)
stocks_close=pd.concat([df_BTC['close'],df_SOL['close'],df_XRP['close'],df_DOGE['close']],axis=1)
stocks_returns.columns=['KRW-BTC','KRW-SOL','KRW-XRP','KRW-DOGE']
stocks_close.columns=['KRW-BTC','KRW-SOL','KRW-XRP','KRW-DOGE']

returns_corr=stocks_returns.corr()
close_corr=stocks_close.corr()


fig=make_subplots(rows=2, cols=1, vertical_spacing=0.2)
fig.add_trace(go.Heatmap(
                    z=returns_corr,
                    x=['BTC','SOL','XRP','DOGE'],
                    y=['BTC','SOL','XRP','DOGE'],
                    text=round(returns_corr,3),
                    texttemplate="%{text}",
                    textfont={"size":20}),row=1,col=1)
fig.add_trace(go.Heatmap(
                    z=close_corr,
                    x=['BTC','SOL','XRP','DOGE'],
                    y=['BTC','SOL','XRP','DOGE'],
                    text=round(close_corr,3),
                    texttemplate="%{text}",
                    textfont={"size":20}),row=2,col=1)

fig.update_yaxes(title_text=f'returns_corr', row=1, col=1)
fig.update_yaxes(title_text=f'close_corr', row=2, col=1)
fig.show()

## KRW-BTC,SOL,XRP,DOGE risk and returns
     - scatter 

In [20]:
fig=go.Figure()
for i in stocks_returns.columns:
    fig.add_trace(go.Scatter(x=[stocks_returns[i].mean()],
                                y=[stocks_returns[i].std()],
                                mode='markers',
                                name=i
                                )) # hover text goes here

fig.update_layout(title='Risk-Returns')
fig.update_yaxes(title_text=f'risk')
fig.update_xaxes(title_text=f'returns')
fig.show()

## Predicting closing price KRW-BTC
      - LSTM

In [41]:
df_BTC=applytech(api.day_candle(market="KRW-BTC",count=1500))
df_BTC.sort_index(inplace=True)
data = df_BTC.filter(['close','open','high','low','Daily return'])
dataset = data.values
training_data_len = int(np.ceil( len(dataset) * .9 ))
training_data_len


1350

In [42]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

scaled_data

array([[0.01370462, 0.01414209, 0.01476935, 0.01376513, 0.33363817],
       [0.0098735 , 0.0138721 , 0.01320003, 0.01004907, 0.20301534],
       [0.01077342, 0.00993803, 0.01147883, 0.00939483, 0.29900454],
       ...,
       [0.28620282, 0.27232522, 0.28495855, 0.27744848, 0.23026548],
       [0.28562429, 0.28609447, 0.28228817, 0.28635918, 0.21038592],
       [0.289314  , 0.28588877, 0.28874264, 0.28836114,        nan]])

In [43]:
train_data = scaled_data[0:int(training_data_len), :]
print(train_data.shape)
x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, :])
    y_train.append(train_data[i, :])

    

x_train, y_train = np.array(x_train), np.array(y_train)
print(x_train.shape,y_train.shape)
#x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))


(1350, 5)
(1290, 60, 5) (1290, 5)


In [44]:
from keras.models import Sequential
from keras.layers import Dense, LSTM

# Build the LSTM model
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape= (x_train.shape[1], x_train.shape[2])))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(x_train.shape[2]))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(x_train, y_train,batch_size=10,epochs=2)

Epoch 1/2
129/129 [==============================] - 9s 38ms/step - loss: 0.0074
Epoch 2/2
129/129 [==============================] - 5s 39ms/step - loss: 0.0019


In [49]:
# Create the testing data set
# Create a new array containing scaled values from index 1543 to 2002 
test_data = scaled_data[training_data_len - 60: , :]
# Create the data sets x_test and y_test
x_test = []
y_test = dataset[training_data_len:, :]
for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, :])
    
# Convert the data to a numpy array
x_test = np.array(x_test)
print(x_test.shape)

# Reshape the data
#x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], x_test.shape[2] ))
#print(x_test.shape)
# Get the models predicted price values 
predictions = model.predict(x_test)
print(predictions.shape)
predictions = scaler.inverse_transform(predictions)

# Get the root mean squared error (RMSE)
rmse = np.sqrt(np.mean(((predictions[:-1,:] - y_test[:-1,:]) ** 2)))
rmse

(150, 60, 5)
5/5 [==============================] - 0s 28ms/step
(150, 5)


1096997.831588751

In [47]:
df_BTC_pred=pd.DataFrame(index=df_BTC.index[training_data_len:],columns=['close','open','high','low','Daily return'],data=predictions)


In [48]:
df_BTC_pred

,close,open,high,low,Daily return
datetime64_kst,,,,,
2022-08-20 09:00:00,31165678.0,31150356.0,31911704.0,29766648.0,0.009920
2022-08-21 09:00:00,30847612.0,30701918.0,31532142.0,29480798.0,0.010056
2022-08-22 09:00:00,30486680.0,30233604.0,31129744.0,29147360.0,0.010027
2022-08-23 09:00:00,30120618.0,29792180.0,30750674.0,28807538.0,0.009790
2022-08-24 09:00:00,29765830.0,29393602.0,30393488.0,28469634.0,0.009545
...,...,...,...,...,...
2023-01-12 09:00:00,21184798.0,21476046.0,22329108.0,20284782.0,0.001870
2023-01-13 09:00:00,21438920.0,21776468.0,22681570.0,20530346.0,0.000989
2023-01-14 09:00:00,21812860.0,22226842.0,23155566.0,20880682.0,0.000168


In [50]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    name="Raw Data",
    x=df_BTC.index, y=df_BTC['close'],
    
))
fig.add_trace(go.Scatter(
    name="predicton Data",
     x=df_BTC_pred.index, y=df_BTC_pred['close']
))

fig.update_xaxes(
    rangeslider_visible=True)
fig.show()

In [51]:
fig = go.Figure()
fig.add_trace(go.Candlestick(x=df_BTC.index,
                    open=df_BTC['open'], high=df_BTC['high'],
                    low=df_BTC['low'], close=df_BTC['close'],showlegend=False))
fig.add_trace(go.Candlestick(x=df_BTC_pred.index,
                    open=df_BTC_pred['open'], high=df_BTC_pred['high'],
                    low=df_BTC_pred['low'], close=df_BTC_pred['close'],showlegend=False))

In [52]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    name="Raw Data",
    x=df_BTC.index, y=df_BTC['Daily return'],
    
))
fig.add_trace(go.Scatter(
    name="predicton Data",
     x=df_BTC_pred.index, y=df_BTC_pred['Daily return']
))

fig.update_xaxes(
    rangeslider_visible=True)
fig.show()

In [64]:
acc=pd.DataFrame(pd.concat([df_BTC['Daily return'],df_BTC_pred['Daily return']],axis=1).values,columns=['y','ypred'])
acc=acc.dropna(axis=0)
acc['correct']=acc.apply(lambda x: 1 if ((x.y*x.ypred)>0) else 0,axis=1)
print(acc['correct'].sum()/len(acc))

0.5503355704697986
